<a href="https://colab.research.google.com/github/TaoM1992/gh-exercise/blob/master/ml2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import *
from sklearn.impute import *
from sklearn.compose import *
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance

In [117]:
data = pd.read_csv('train.zip')

In [118]:
data.head()

,problem_id,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,...,v939,v940,v941,v942,v943,v944,v945,v946,v947,v948,v949,v950,v951,v952,v953,v954,v955,v956,v957,v958,v959,v960,v961,v962,v963,v964,v965,v966,v967,v968,v969,v970,v971,v972,v973,v974,v975,v976,v977,target
0,0,0.35,0.36,0.77,0.42,0.48,0.12,0.43,0.49,0.19,0.78,0.38,0.22,0.81,0.32,0.87,0.39,0.61,0.23,0.56,0.86,0.82,0.69,1.00,0.92,0.37,0.33,0.27,0.42,0.51,0.70,0.16,0.44,0.61,0.94,0.48,0.74,0.07,0.44,0.41,...,0.81,0.73,0.80,0.35,0.24,0.42,0.17,0.71,0.69,0.45,0.93,0.97,0.02,0.18,0.02,0.44,0.89,0.99,0.22,0.97,0.95,0.02,0.34,0.57,0.86,0.85,0.62,0.87,0.16,1.00,0.77,0.57,0.56,0.40,0.75,0.08,0.14,0.43,0.88,0
1,0,0.12,0.17,0.38,0.43,0.57,0.09,0.11,0.58,0.35,0.87,0.61,0.41,0.14,0.66,0.15,0.63,0.26,0.88,0.66,0.03,0.03,0.80,0.01,0.23,0.67,0.15,0.28,0.03,0.97,0.67,0.26,0.20,0.64,0.24,0.48,0.01,0.77,0.41,0.22,...,0.31,0.75,0.95,0.20,0.34,0.31,0.11,0.19,0.09,0.14,0.10,0.01,0.02,0.19,0.50,0.26,0.05,0.03,0.26,0.08,0.10,0.03,0.78,0.66,0.01,0.11,0.33,0.27,0.27,0.39,0.07,0.59,0.06,0.17,0.09,0.04,0.04,0.45,0.14,1
2,0,0.67,0.16,0.85,0.41,0.57,0.27,0.83,0.73,0.26,0.20,0.63,0.54,0.41,0.25,0.06,0.71,0.71,0.63,0.36,0.96,0.22,0.89,0.11,0.09,0.42,0.54,0.20,0.18,0.09,0.58,0.98,0.37,0.53,0.06,0.48,0.90,0.02,0.47,0.70,...,0.35,0.20,0.93,0.33,0.16,0.43,0.32,0.16,0.73,0.37,0.78,0.99,0.04,0.20,0.03,0.19,0.24,0.38,0.11,0.79,0.52,0.20,0.81,0.66,0.87,0.16,0.53,0.64,0.16,0.65,0.24,0.42,0.57,0.58,0.37,0.01,0.10,0.38,0.06,1
3,0,0.70,0.20,0.62,0.41,0.41,0.10,0.80,0.52,0.82,0.01,0.66,0.19,0.81,0.26,0.91,0.47,0.68,0.29,0.65,0.03,0.93,0.92,0.06,0.08,0.11,0.09,0.32,0.62,0.75,0.69,0.74,0.14,0.73,0.14,0.48,0.21,0.02,0.48,0.06,...,0.58,0.34,0.90,0.63,0.08,0.51,0.63,0.36,0.88,0.57,0.12,1.00,0.06,0.57,0.01,0.19,0.94,0.85,0.80,0.99,0.98,0.79,0.41,0.56,0.82,0.83,0.55,0.88,0.26,0.96,0.04,0.52,0.25,0.07,0.23,0.04,0.76,0.41,0.59,1
4,0,0.72,0.75,0.74,0.42,0.41,0.76,0.34,0.72,0.06,0.87,0.28,0.50,0.55,0.53,0.68,0.73,0.77,0.27,0.55,0.02,0.88,0.74,1.00,0.06,0.16,0.18,0.31,0.53,0.79,0.64,0.10,0.60,0.67,0.06,0.48,0.14,0.24,0.64,0.15,...,0.75,0.58,0.33,0.67,0.87,0.30,0.22,0.56,0.77,0.65,0.90,0.96,0.05,0.41,0.01,0.34,0.05,0.89,0.88,0.86,0.87,0.77,0.65,0.63,0.83,0.93,0.59,0.38,0.28,0.99,0.81,0.41,0.16,0.03,0.21,0.10,0.12,0.53,0.30,1


### EDA

In [4]:
data.describe()

,problem_id,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,...,v939,v940,v941,v942,v943,v944,v945,v946,v947,v948,v949,v950,v951,v952,v953,v954,v955,v956,v957,v958,v959,v960,v961,v962,v963,v964,v965,v966,v967,v968,v969,v970,v971,v972,v973,v974,v975,v976,v977,target
count,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,...,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000,8302.000000
mean,10.340520,0.503570,0.502644,0.498356,0.505302,0.524481,0.509112,0.515471,0.501418,0.502067,0.508455,0.508628,0.503551,0.522857,0.513729,0.504331,0.500944,0.504099,0.503487,0.507472,0.505976,0.507056,0.497909,0.502781,0.510805,0.496324,0.495556,0.499857,0.505316,0.510243,0.499392,0.501517,0.511441,0.500809,0.498538,0.499953,0.500846,0.507916,0.494775,0.503457,...,0.504060,0.500855,0.517391,0.498129,0.510248,0.515057,0.521003,0.511013,0.507489,0.503304,0.507772,0.506633,0.503946,0.506692,0.506509,0.504409,0.496615,0.517257,0.500699,0.513926,0.508292,0.500965,0.498086,0.504705,0.513251,0.499975,0.505491,0.508988,0.501020,0.507587,0.513447,0.504952,0.506437,0.495670,0.498960,0.517119,0.503209,0.501382,0.506916,0.659359
std,6.183559,0.267315,0.271960,0.273268,0.273052,0.363815,0.326855,0.319661,0.248639,0.312049,0.343824,0.352683,0.326892,0.330060,0.325781,0.249905,0.252319,0.228804,0.286075,0.287020,0.317533,0.275351,0.245943,0.311828,0.305258,0.271317,0.225880,0.219016,0.288524,0.333755,0.281109,0.299889,0.265093,0.272141,0.283707,0.295192,0.286205,0.311652,0.274326,0.292269,...,0.296189,0.304701,0.355838,0.268918,0.317842,0.306316,0.317508,0.335985,0.315008,0.284774,0.320874,0.322537,0.285278,0.328904,0.334018,0.239582,0.305747,0.377786,0.315483,0.304076,0.321097,0.256503,0.288095,0.305139,0.330490,0.227577,0.276869,0.305742,0.290328,0.297883,0.335301,0.314542,0.285526,0.295917,0.252681,0.276100,0.308813,0.278673,0.294042,0.782339
min,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,...,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.000000
25%,5.000000,0.290000,0.280000,0.270000,0.280000,0.140000,0.200000,0.220000,0.300000,0.210000,0.170000,0.150000,0.190000,0.210000,0.190000,0.320000,0.300000,0.310000,0.240000,0.270000,0.210000,0.270000,0.320000,0.220000,0.220000,0.270000,0.330000,0.320000,0.270000,0.190000,0.280000,0.240000,0.290000,0.270000,0.240000,0.240000,0.250000,0.220000,0.270000,0.240000,...,0.230000,0.220000,0.160000,0.280000,0.200000,0.260000,0.230000,0.170000,0.210000,0.250000,0.190000,0.200000,0.26

At first glance, all numerical variables except prolem_id seem to have 
similar distribution. 

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8302 entries, 0 to 8301
Columns: 980 entries, problem_id to target
dtypes: float64(978), int64(2)
memory usage: 62.1 MB


In [34]:
data.problem_id.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

We see actually we can treat problem_id as a categorical variable. 

### Feature Engineering

In [41]:
# convert problem_id into categorical variable 
data.problem_id = data.problem_id.astype('category')

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8302 entries, 0 to 8301
Columns: 980 entries, problem_id to target
dtypes: category(1), float64(978), int64(1)
memory usage: 62.0 MB


In [119]:
X = data.drop('target', axis=1)
y = data.target

In [68]:
# one-hot encoding problem_id
categorical_columns = (X.dtypes == 'category')
cat_pipe = Pipeline([('ohe', OneHotEncoder())])
preprocessing = ColumnTransformer([('categorical', cat_pipe, categorical_columns)])

In [81]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)

Let's first see feature importance with a simple xgboost model without any fine tuning.


### Feature Importance

In [ ]:
# list for important columns
imp_cols = []
def importance(model, X_columns, X_val, y_val):
    """Compute the important columns and add them to the list imp_cols"""
    r = permutation_importance(model, X_val, y_val,
                           n_repeats=3)
    for i in r.importances_mean.argsort()[::-1]:
        if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#             print(f"{X_columns[i]:<20}"
#                   f"{r.importances_mean[i]:.3f}"
#                   f" +/- {r.importances_std[i]:.3f}")
            imp_cols.append(X_columns[i])
importance(clf, X.columns, X_val, y_val)
# run model only using important columns
# train val split
X = train.iloc[:,:-1].drop('problem_id', 1)[imp_cols]
y = train['target'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
# fit model
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)
# predict
y_proba = clf.predict_proba(X_val)
y_pred = clf.predict(X_val)
# validation log loss
val_log_loss = log_loss(y_val, y_proba)
print(f'Validation Log Loss: {val_log_loss:.4f}') 
# validation accuracy
val_accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {val_accuracy:.4f}') 

### Fine Tuning with Grid Search

In [70]:
hyper = {'clf__loss': ['deviance', 'exponential'],
         'clf__learning_rate': [0.01, 0.1],
         'clf__n_estimators': [50, 100],
         'clf__max_depth':[5, 10],
         'clf__max_features':['auto', 'sqrt', 'log2']
}

In [71]:
pipe = Pipeline([('preprocessing', preprocessing), 
                 ('clf', GradientBoostingClassifier())])

In [72]:
clf_grid = GridSearchCV(pipe,
                        hyper,
                        cv=3,
                        n_jobs=-1,
                        verbose=True)

In [73]:
clf_grid.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:    5.7s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('categorical',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('ohe',
                                                                                          OneHotEncoder(categories='auto',
                                                                                                        drop=None,
                                 

In [75]:
print(clf_grid.best_estimator_.get_params()['clf'])

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features='auto', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [82]:
pred = clf_grid.best_estimator_.predict(X_val)
accuracy_score(y_val, pred)

0.5833333333333334

We see a very low accuracy with problem_id as categorical variable. Let's try to drop it

In [120]:
X = X.drop('problem_id', axis=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)

In [121]:
hyper1 = {'loss': ['deviance', 'exponential'],
         'learning_rate': [0.01, 0.1],
         'n_estimators': [50, 100],
         'max_depth':[5, 10],
         'max_features':['auto', 'sqrt', 'log2']
}

In [122]:
clf_grid1 = GridSearchCV(GradientBoostingClassifier(),
                        hyper1,
                        cv=3,
                        n_jobs=-1,
                        verbose=True)

In [123]:
clf_grid1.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 17.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_c...
                 

In [124]:
print(clf_grid1.best_estimator_.get_params())

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


In [126]:
pred1 = clf_grid1.best_estimator_.predict(X_val)
accuracy_score(y_val, pred1)

0.7027938342967245

Validation accuracy increased dramatically. Let's try to treat "problem_id" as numerical variable.

In [128]:
X = data.drop('target', axis=1)
y = data.target
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)

In [129]:
clf_grid2 = GridSearchCV(GradientBoostingClassifier(),
                        hyper1,
                        cv=3,
                        n_jobs=-1,
                        verbose=True)

In [130]:
clf_grid2.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed: 16.9min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_c...
                 

In [131]:
pred2 = clf_grid2.best_estimator_.predict(X_val)
accuracy_score(y_val, pred2)

0.7133911368015414

We see that "problem_id" helped to increase accuracy on validation set. Let's try PCA with our best model.

In [133]:
# try to use PCA to select numerical variables
pipe_pca = Pipeline([('PCA', PCA(n_components = 'mle')),
                     ('clf', clf_grid2.best_estimator_)])

In [134]:
pipe_pca.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('PCA',
                 PCA(copy=True, iterated_power='auto', n_components='mle',
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('clf',
                 GradientBoostingClassifier(ccp_alpha=0.0,
                                            criterion='friedman_mse', init=None,
                                            learning_rate=0.1, loss='deviance',
                                            max_depth=5, max_features='auto',
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            

In [135]:
pred3 = pipe_pca.predict(X_val)
accuracy_score(y_val, pred3)

0.6931599229287091

### Test

In [140]:
test = pd.read_csv('test0.csv')
X_test = test.drop(['target','problem_id'], axis=1)
X_test = X_test.set_index('obs_id')

In [141]:
pred_test = clf_grid1.best_estimator_.predict(X_test)

In [142]:
test.target = pred_test

In [143]:
test_final = test[['obs_id', 'target']]

In [144]:
test_final = test_final.set_index('obs_id')

In [145]:
test_final.to_csv('sample_data/test_final_1.csv')

In [146]:
test_final

,target
obs_id,
0,1
1,1
2,1
3,0
4,1
...,...
2036,0
2037,0
2038,0
